<a href="https://colab.research.google.com/github/ma850419/Various_scripts/blob/main/video_archeology.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import ee
ee.Authenticate()
ee.Initialize(project='velvety-ring-328419')

In [8]:

import geemap
import cv2
import os
import matplotlib.pyplot as plt


In [ ]:
!pip install geedim

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.1/74.1 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 76.2 MB/s eta 0:00:00


In [9]:
import geemap
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

# Initialize Earth Engine

# Area of Interest and time range
aoi = ee.Geometry.BBox(-74.1, -10.5, -73.5, -10.0)
start, end = '2022-07-01', '2022-07-31'
scale = 30

# Datasets
s2 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED').filterBounds(aoi).filterDate(start, end).median()
ndvi = s2.normalizedDifference(['B8', 'B4']).rename('NDVI')

s1 = ee.ImageCollection('COPERNICUS/S1_GRD').filterBounds(aoi).filterDate(start, end).select('VV').median().rename('Radar')
srtm = ee.Image('USGS/SRTMGL1_003').clip(aoi).rename('Elevation')

lst = ee.ImageCollection('MODIS/061/MOD11A2').filterBounds(aoi).filterDate(start, end) \
    .mean().select('LST_Day_1km').multiply(0.02).subtract(273.15).rename('LST')

smap = ee.ImageCollection('NASA_USDA/HSL/SMAP10KM_soil_moisture').filterBounds(aoi).filterDate(start, end) \
    .select('ssm').mean().rename('Soil_Moisture')

# Map of image layers
raw_layers = {
    'NDVI': ndvi,
    'Radar': s1,
    'Elevation': srtm,
    'LST': lst,
    'Soil_Moisture': smap
}

# Colormaps for styling
colormaps = {
    'NDVI': 'Greens',
    'Radar': 'gray',
    'Elevation': 'viridis',
    'LST': 'coolwarm',
    'Soil_Moisture': 'Reds'
}
frames=[]
# Generate images
for name, img in raw_layers.items():
    array = geemap.ee_to_numpy(img, region=aoi, scale=scale)
    array = np.squeeze(array)

    if np.all(np.isnan(array)):
        print(f"Skipping {name} due to missing data.")
        continue

    norm = (array - np.nanmin(array)) / (np.nanmax(array) - np.nanmin(array))
    norm = np.nan_to_num(norm)

    cmap = plt.get_cmap(colormaps[name])
    rgb = (cmap(norm)[:, :, :3] * 255).astype(np.uint8)

    Image.fromarray(rgb).save(f"{name}.png")
    print(f"Saved {name}.png")
    # Add to video frames
    frame = cv2.imread(f"{name}.png")
    frames.append(frame)


Saved NDVI.png
Saved Radar.png
Saved Elevation.png
Saved LST.png
Saved Soil_Moisture.png


In [10]:
height, width, _ = frames[0].shape
video = cv2.VideoWriter('archaeology_layers.mp4', cv2.VideoWriter_fourcc(*'mp4v'), 1, (width, height))

for frame in frames:
    video.write(frame)

video.release()
